<a href="https://colab.research.google.com/github/pinak139/Temp/blob/main/Network_Planning_V_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install pulp pyomo simpy
#!pip install highspy
#!pip install pandas
!pip install openpyxl
!pip install polars
!pip install cplex

In [ ]:
import pulp as plp
import pandas as pd
import polars as plr
from docplex.mp.model import Model

In [ ]:
# Get Solver
solver_list = plp.listSolvers(onlyAvailable=True)
solver = plp.getSolver('CPLEX_CMD')

In [ ]:
location = pd.read_excel('Julio_Data_Template_V3_0.xlsx',sheet_name = 'LOCATION')
product = pd.read_excel('Julio_Data_Template_V3_0.xlsx',sheet_name = 'PRODUCT')
#product = product[product['pCost'] > 350]
products = list(product['ProductID'].unique())
locationProduct = pd.read_excel('Julio_Data_Template_V3_0.xlsx',sheet_name = 'LOCATIONPRODUCT')
locationProduct = locationProduct[locationProduct['ProductID'].isin(products)]
supplier = pd.read_excel('Julio_Data_Template_V3_0.xlsx',sheet_name = 'SUPPLIER')
supplierProduct = pd.read_excel('Julio_Data_Template_V3_0.xlsx',sheet_name = 'SUPPLIERPRODUCT')
supplierProduct = supplierProduct[supplierProduct['pProductID'].isin(products)]
openpo = pd.read_excel('Julio_Data_Template_V3_0.xlsx',sheet_name = 'OPENPO')
openpo = openpo[openpo['ixSKU'].isin(products)]
opensto = pd.read_excel('Julio_Data_Template_V3_0.xlsx',sheet_name = 'OPENSTO')
opensto = opensto[opensto['ixSKU'].isin(products)]
currentinv = pd.read_excel('Julio_Data_Template_V3_0.xlsx',sheet_name = 'CURRENTINVENTORY')
currentinv = currentinv[currentinv['ixSKU'].isin(products)]


forecast = pd.read_excel('Julio_Data_Template_V3_0.xlsx',sheet_name = 'FORECAST')
forecast = forecast[forecast['ixSKU'].isin(products)]

calendar = pd.read_excel('Julio_Data_Template_V3_0.xlsx',sheet_name = 'CALENDAR')
calmondays = pd.read_excel('Julio_Data_Template_V3_0.xlsx',sheet_name = 'CALENDAR_MONTH_DAYS')
alloc = pd.read_excel('Julio_Data_Template_V3_0.xlsx',sheet_name = 'ALLOC')
alloc = alloc[alloc['ixSKU'].isin(products)]

temp = calendar[['MONTHSTARTDATE','FISCAL_DATE']].merge(forecast, left_on = 'MONTHSTARTDATE',right_on = 'ForecastedPeriod', how = 'inner')
temp = temp.merge(calmondays, on = 'MONTHSTARTDATE', how = 'inner')
temp['DailyForecast'] = temp['Forecast']/temp['NBR_DAYS_IN_MONTH']
forecast = temp[['FISCAL_DATE','ixSKU','DailyForecast']]

locationProduct['nodeID'] = list(zip(locationProduct.LocationID, locationProduct.ProductID))
currentinv['nodeID'] = list(zip(currentinv.ixLocation, currentinv.ixSKU))
opensto['nodeID'] = list(zip(opensto.ixToLocation, opensto.ixSKU))
internal_node = locationProduct[['LocationID','ProductID','nodeID','pPrimarySupplierID','nLeadTime','nWACC','nInvTgtDays','nInvMinDays','nLeadTime']].merge(currentinv[['nodeID','ixSKU','nInitInv']], on = 'nodeID', how = 'left')

central_node = internal_node[internal_node['LocationID'] == 99]
sat_locs = [25,85]
sat_node =  internal_node[internal_node['LocationID'].isin(sat_locs)]

opensto = opensto.drop_duplicates()


central_node = central_node.merge(openpo, on = 'ixSKU', how = 'left').fillna(0)
#sat_node = internal_node[internal_node['LocationID'].isin(sat_locs)]
sat_node = sat_node.merge(opensto, on = 'nodeID',how = 'left')
sat_node['ProductID'] = sat_node['ixSKU_x']
sat_node = sat_node[['LocationID','ProductID','nodeID','nLeadTime','nInvTgtDays','nInvMinDays','nInitInv','iQuantityRequested']]
sat_node = sat_node.fillna(0)


# Data Prep




alloc = alloc[['ixSKU','Location','DailyDemand']].merge(location[['LocShort','LocationID']], left_on = 'Location', right_on = 'LocShort')

node_forecast = forecast.merge(alloc, on = 'ixSKU', how = 'inner')
node_forecast['NodeDailyForecast'] = node_forecast['DailyForecast']* node_forecast['DailyDemand']
node_forecast = node_forecast[['FISCAL_DATE','ixSKU','LocationID','NodeDailyForecast']]

product['ProductID'] = product['ProductID'].astype(str)
locationProduct['ProductID'] = locationProduct['ProductID'].astype(str)
supplierProduct['pProductID'] = supplierProduct['pProductID'].astype(str)
openpo['ixSKU'] = openpo['ixSKU'].astype(str)
opensto['ixSKU'] = opensto['ixSKU'].astype(str)
currentinv['ixSKU'] = opensto['ixSKU'].astype(str)
forecast['ixSKU'] = forecast['ixSKU'].astype(str)
alloc['ixSKU'] = alloc['ixSKU'].astype(str)



# Internal Nodes

central_node['InvOO'] = central_node['OpenPOQty']
#central_node[''] = central_node['OpenPOQty']
central_node = central_node[['nodeID','LocationID','ProductID','pPrimarySupplierID','nLeadTime','nWACC','nInvTgtDays','nInvMinDays','nInitInv','InvOO']]
central_node['nodeType'] = 'central'



sat_node['pPrimarySupplierID'] = 99
#sat_node['ixSKU'] = sat_node['ixSKU_x']
sat_node['InvOO'] = sat_node['iQuantityRequested']
sat_node['nWACC'] = 0.08

sat_node = sat_node[['nodeID','LocationID','ProductID','pPrimarySupplierID','nLeadTime','nWACC','nInvTgtDays','nInvMinDays','nInitInv','InvOO']]
sat_node['nodeType'] = 'satellite'

node = pd.concat([central_node,sat_node])


#############


#Periods

periods = list(node_forecast['FISCAL_DATE'].unique())
numPeriods = range(1,len(periods) + 1)
period_dict =  {'periodID': numPeriods, 'FISCAL_DATE': periods}
period_df = pd.DataFrame(period_dict)

# Forecast

from datetime import datetime,timedelta
def first_day_of_week(date):
    return date - timedelta(days = date.weekday())

period_df['WeekStart'] = period_df['FISCAL_DATE'].apply(first_day_of_week)
numWks = len(list(period_df['WeekStart'].unique()))
Weeks = list(range(1,numWks + 1))

node_forecast = node_forecast.merge(period_df, on  = 'FISCAL_DATE')
node_forecast = node_forecast.groupby(['WeekStart','ixSKU','LocationID']).sum('NodeDailyForecast').reset_index()

timep = list(period_df['WeekStart'].unique())
periods = pd.DataFrame({'WeekStart': timep,'weekID': Weeks})

weekforecast = node_forecast.merge(periods, on = 'WeekStart')
weekforecast['forecastID'] = list(zip(weekforecast.LocationID,weekforecast.ixSKU,weekforecast.weekID))



# Forecast Prep

weekforecast = weekforecast.set_index('forecastID')
weekforecast['nodeID'] = list(zip(weekforecast.LocationID, weekforecast.ixSKU))
dWeekOut = weekforecast['weekID'].to_dict()
dWeekStart = weekforecast['WeekStart'].to_dict()
dProduct = weekforecast['ixSKU'].to_dict()
dLocation = weekforecast['LocationID'].to_dict()
dForecast = weekforecast['NodeDailyForecast'].to_dict()
dDayOut = weekforecast['periodID'].to_dict()
dNode = weekforecast['nodeID'].to_dict()

# Periods

periods = list(weekforecast['weekID'].unique())
#periods = periods[:9]


# Location MAster

location = location.set_index('LocationID')
lLocShort = location['LocShort'].to_dict()
lCity = location['City'].to_dict()
lState = location['State'].to_dict()



# Product Master

product = product.set_index('ProductID')
pName = product['pProductName'].to_dict()
pPrice = product['pPrice'].to_dict()
pVolume = product['pVolume'].to_dict()
pMOQ = product['pMOQ'].to_dict()
pCost = product['pCost'].to_dict()
pLeadTime = product['pLeadTime'].to_dict()
pPrimarySupplierID = product['pPrimarySupplierID'].to_dict()
pPrimarySupplierName = product['pPrimarySupplierName'].to_dict()


# Vendor Master

supplier = supplier.set_index('sSupplierID')
sVendorName = supplier['sSupplierName'].to_dict()
sCity = supplier['sSupplierCity'].to_dict()
sState = supplier['sSupplierState'].to_dict()
sPaymentTerms = supplier['sPaymentTerms'].to_dict()
sFreightTerms = supplier['sFreightTerms'].to_dict()
sOverseas = supplier['flgOverseas'].to_dict()
sMinPODollars = supplier['mMinPODollars'].to_dict()
sMinFreightDollars = supplier['mMinFreightDollars'].to_dict()


# Vendor Nodes


supplierProduct['supplierProductID'] = list(zip(supplierProduct.pVendorID,supplierProduct.pProductID))
supplierProduct =  supplierProduct.set_index('supplierProductID')
spCost = supplierProduct['mCost'].to_dict()



sat_node = sat_node.set_index('nodeID')
central_node = central_node.set_index('nodeID')
node = node.set_index('nodeID')
nLocation = node['LocationID'].to_dict()
nProduct = node['ProductID'].to_dict()
nVendor = node['pPrimarySupplierID'].to_dict()
nLeadTime = node['nLeadTime'].to_dict()
nWACC = node['nWACC'].to_dict()
nInvTgtDays = node['nInvTgtDays'].to_dict()
nInvMinDays = node['nInvMinDays'].to_dict()
nInvOO = node['InvOO'].to_dict()
nType = node['nodeType'].to_dict()
nInitInv = node['nInitInv'].to_dict()

# Index Sets

locations  = list(location.index)
products = list(product.index)

nodes = list(node.index)
sat_nodes = list(node[node['nodeType'] == 'satellite'].index)
central_nodes = list(node[node['nodeType'] == 'central'].index)
suppliers = list(supplier.index)


# ForecastID
forecasts = list(weekforecast.index)

In [ ]:
# Decision Variables

from pulp import *

# Decision Variables

model = LpProblem('JULIO_V_1_1: General Sourcing andn Distribution Model', LpMinimize)

# CORE VARIABLES

INV = LpVariable.dicts("InventoryOnHand", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
FCST = LpVariable.dicts("Forecast", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
TSREQ = LpVariable.dicts("TransferReq", [(l,p,t) for l,p,t in forecasts ] , lowBound = 0, cat = 'Continuous')
TSREQm = LpVariable.dicts("TransferReq", [(l,p,t) for l,p,t in forecasts] , lowBound = 0, cat = 'Continuous')
TSREC = LpVariable.dicts("TransferRec", [(l,p,t) for l,p,t in forecasts] , lowBound = 0, cat = 'Continuous')
DEPREQ = LpVariable.dicts("DependentReq", [(l,p,t) for l,p,t in forecasts] , lowBound = 0, cat = 'Continuous')
PREQ = LpVariable.dicts("PurchaseReq", [(l,p,t)  for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
PREC = LpVariable.dicts("PurchaseRec", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
STOO = LpVariable.dicts("StockTransferOnOrder", [(l,p,t) for  l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
POO = LpVariable.dicts("PurchaseOnOrder", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
DEL = LpVariable.dicts("Delivered", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')

# Calc Variables
BAS = LpVariable.dicts("BaseStk", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
SAF = LpVariable.dicts("SafStk", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
OUL = LpVariable.dicts("SafStk", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
# Slack Variables

MIS = LpVariable.dicts("MissedSales", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
INVm = LpVariable.dicts("Invor p in products for t in periodsentoryMinSlack", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
INVp = LpVariable.dicts("InventoryMaxSlack", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
BUYm = LpVariable.dicts("PurchaseMinSlack", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
BUYp = LpVariable.dicts("PurchaseMaxSlack", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')

AGGm = LpVariable.dicts("MinVendorSlack", [(s,t) for s in nodes for t in periods], lowBound = 0, cat = 'Continuous')
AGGm = LpVariable.dicts("MinVendorSlack", [(s,t) for s in nodes for t in periods], lowBound = 0, cat = 'Continuous')


In [ ]:
nbPeriods = 46

#dForecast

for l,p,t in forecasts:
    if t < nbPeriods:
        model += FCST[l,p,t] == dForecast[l,p,t]


for l,p,t in forecasts:
    if t < nbPeriods:
        if nType[l,p] == 'satellite':
            model += BAS[l,p,t] == lpSum(FCST[l,p,ts] for ts in periods if ts >= t and ts < t + 4 )


for l,p,t in forecasts:
    if t < nbPeriods:
        model += OUL[l,p,t] == lpSum(dForecast[l,p,ts] for ts in periods if ts >= t and ts < t + 6 )


for l,p,t in forecasts:
    if nType[l,p] == 'satellite':
        if t < 5:
            model += STOO[l,p,t] == nInvOO[l,p] + lpSum(TSREC[l,p,ts] for ts in periods if ts <=t and ts > t - 4)
        if t >= 5:
            model += STOO[l,p,t] == lpSum(TSREC[l,p,ts] for ts in periods if ts <=t and ts > t - 4)

#for l,p,t in forecasts:
#    if nType[l,p] == 'satellite':
#        model += TSREQ[l,p,t] == OUL[l,p,t] - (INV[l,p,t] + STOO[l,p,t])

for l,p,t in forecasts:
    if nType[l,p] == 'satellite':
        if t == 1:
            INV[l,p,t] == nInitInv[l,p] + TSREC[l,p,t] - FCST[l,p,t]
        if t > 1:
            INV[l,p,t] == INV[l,p,t - 1] + TSREC[l,p,t] - FCST[l,p,t]


for l,p,t in forecasts:
    if nType[l,p] == 'satellite':
        model += TSREQ[l,p,t] >= OUL[l,p,t] - (INV[l,p,t] + STOO[l,p,t])


for l,p,t in forecasts:
    if nType[l,p] == 'satellite':
        if t < nbPeriods:
            model += TSREC[l,p,t+4] == TSREQ[l,p,t]


for l,p,t in forecasts:
    if t < nbPeriods:
        if nType[l,p] == 'central':
            model += DEPREQ[l,p,t] == lpSum(TSREQ[ls,p,t] for ls in locations if  nVendor[ls,p] == l)
        if nType[l,p] == 'satellite':
            model += DEPREQ[l,p,t] == 0



for l,p,t in forecasts:
  if t < nbPeriods:
      if nType[l,p] == 'central':
          model += OUL[l,p,t] == lpSum(DEPREQ[l,p,ts] for ts in periods if ts >= t and ts < t + 4 )\
                              + lpSum(FCST[l,p,ts] for ts in periods if ts >= t and ts < t + 4 )



for l,p,t in forecasts:
    if nType[l,p] == 'central':
        if t < 5:
            model += POO[l,p,t] == nInvOO[l,p] + lpSum(PREC[l,p,ts] for ts in periods if ts <=t and ts > t - 4)
        if t >= 5:
            model += POO[l,p,t] == lpSum(PREC[l,p,ts] for ts in periods if ts <=t and ts > t - 4)


for l,p,t in forecasts:
    if nType[l,p] == 'central':
        model += PREQ[l,p,t] == OUL[l,p,t] - (INV[l,p,t] + POO[l,p,t])


for l,p,t in forecasts:
    if nType[l,p] == 'central':
        if t < nbPeriods:
            model += PREC[l,p,t+4] == PREQ[l,p,t]

for l,p,t in forecasts:
    if nType[l,p] == 'central':
        if t == 1:
            INV[l,p,t] == nInitInv[l,p] + PREC[l,p,t] - FCST[l,p,t]
        if t > 1:
            INV[l,p,t] == INV[l,p,t - 1] + PREC[l,p,t] - FCST[l,p,t]


In [ ]:
model+= 10


model.solve()

print("Status", LpStatus[model.status])